<a href="https://colab.research.google.com/github/tomo0530/CS699_Term_Project/blob/master/CS699_Term_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict hits in Japanese Baseball

# Initialization

Mount google drive for file storage.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install pycaret

In [ ]:
!pip install pycaret

Enable pycaret on colab

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

Install some packages

In [ ]:
import numpy as np
import pandas as pd
from pandas.core.common import random_state
from IPython.display import display, HTML, Markdown
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import VarianceThreshold
from pycaret.classification import setup
from pycaret.classification import compare_models
from pycaret.classification import create_model
from pycaret.classification import tune_model
from pycaret.classification import plot_model
from pycaret.classification import evaluate_model
from pycaret.classification import finalize_model
from pycaret.classification import predict_model

Download pre-processed data

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/tomo0530/CS699_Term_Project/master/baseball_info.csv')
df.head()

,totalPitchingCount,B,S,O,b1,b2,b3,speed,ballX,y,inning_tb,batter_isPitcher,bottomTeam,topTeam,minute,hour,day,month,year,day_of_week,day_of_year,pitcher_team,batter_team,pitcher_order,batter_order,pitcher_ce,pitcherHand_ce,batter_ce,batterHand_ce,pitchType_ce,ballPositionLabel_ce,ballY_ce,inning_number_ce,place_ce,pitcher_te,pitcherHand_te,batter_te,batterHand_te,pitchType_te,ballPositionLabel_te,ballY_te,inning_number_te,place_te
0,1,0,0,0,False,False,False,149,17,0,0,0,9,7,0,18,19,6,2020,4,171,9,7,1,1,207,5950,161,10767,8377,2173,1610,2396,2897,0.038710,0.056187,0.085271,0.060954,0.057915,0.034130,0.032457,0.054026,0.064783
1,2,1,0,0,False,False,False,149,14,0,0,0,9,7,0,18,19,6,2020,4,171,9,7,1,1,207,5950,161,10767,8377,2173,1678,2396,2897,0.055901,0.058406,0.108527,0.060659,0.060290,0.035469,0.060606,0.057351,0.069172
2,3,1,1,0,False,False,False,137,8,0,0,0,9,7,0,18,19,6,2020,4,171,9,7,1,1,207,5950,161,10767,1376,1740,1509,2396,2897,0.064516,0.058069,0.123077,0.060981,0.046406,0.030679,0.067937,0.054382,0.069991
3,4,2,1,0,False,False,False,138,21,0,0,0,9,7,0,18,19,6,2020,4,171,9,7,1,1,207,5950,161,10767,3684,1847,2581,2396,2897,0.056818,0.058403,0.126050,0.060592,0.049750,0.070802,0.095745,0.054703,0.064419
4,5,2,2,0,False,False,False,136,7,1,0,0,9,7,0,18,19,6,2020,4,171,9,7,1,1,207,5950,161,10767,1376,3429,2800,2396,2897,0.038710,0.056187,0.085271,0.060954,0.045045,0.065930,0.103989,0.054026,0.064783


Split data into train and test

In [8]:
df_train, df_test = train_test_split(df, test_size=0.34, random_state=123, stratify=df['y'])
X_train, y_train = df_train.drop('y', axis=1), df_train['y'].values
X_test, y_test = df_test.drop('y', axis=1), df_test['y'].values

Save train and test data

In [26]:
df_train.to_csv('/content/drive/MyDrive/CS699_TermProject/train_data.csv', index=False)
df_test.to_csv('/content/drive/MyDrive/CS699_TermProject/test_data.csv', index=False)

# Classification

## All attributes

Setup pycaret with all attributes

In [19]:
clf = setup(data=df_train, target='y', session_id=123, 
            categorical_features = ['ballX', 'pitcher_ce', 'batter_ce'], silent = True)

,Description,Value
0,session_id,123
1,Target,y
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(13463, 43)"
5,Missing Values,False
6,Numeric Features,10
7,Categorical Features,32
8,Ordinal Features,False
9,High Cardinality Features,False


Compare all classification algorithms.    
Sorting by MCC because "1" (hit) is extremely low.

In [ ]:
best_model = compare_models(sort='MCC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9500,0.7797,0.1163,1.0000,0.2068,0.1975,0.3293,3.363
rf,Random Forest Classifier,0.9452,0.7534,0.0319,0.9000,0.0611,0.0580,0.1596,2.622
dt,Decision Tree Classifier,0.9048,0.5704,0.1933,0.1796,0.1854,0.1352,0.1356,1.009
lda,Linear Discriminant Analysis,0.9383,0.7638,0.0733,0.3076,0.1164,0.0974,0.1253,1.543
lightgbm,Light Gradient Boosting Machine,0.9434,0.7697,0.0376,0.4467,0.0687,0.0615,0.1169,0.591
nb,Naive Bayes,0.2503,0.5669,0.9212,0.0654,0.1220,0.0184,0.0754,0.127
lr,Logistic Regression,0.9433,0.7595,0.0131,0.3167,0.0252,0.0222,0.0581,7.230
svm,SVM - Linear Kernel,0.9164,0.0000,0.0943,0.0533,0.0558,0.0390,0.0455,1.709
qda,Quadratic Discriminant Analysis,0.8485,0.5063,0.1204,0.0528,0.0719,0.0023,0.0038,1.052
ada,Ada Boost Classifier,0.9424,0.7398,0.0019,0.0333,0.0036,0.0012,0.0021,1.674


Tune the best algorithm (Extra Trees Classifier).     
Evaluate the training result by some plots.

In [20]:
et = create_model("et")
tuned_et = tune_model(et, optimize='MCC')
evaluate_model(tuned_et)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6034,0.7887,0.8491,0.1095,0.1940,0.1048,0.2034
1,0.6172,0.7599,0.7963,0.1094,0.1924,0.1020,0.1897
2,0.6087,0.7600,0.7963,0.1072,0.1890,0.0980,0.1850
3,0.6108,0.7537,0.8519,0.1136,0.2004,0.1106,0.2103
4,0.5892,0.7605,0.8491,0.1061,0.1887,0.0985,0.1958
5,0.5531,0.6882,0.7170,0.0856,0.1529,0.0582,0.1202
6,0.6369,0.7422,0.7547,0.1084,0.1896,0.1011,0.1816
7,0.6030,0.7426,0.8868,0.1133,0.2009,0.1123,0.2195
8,0.6083,0.7553,0.8491,0.1108,0.1961,0.1072,0.2061
9,0.5924,0.7537,0.8679,0.1087,0.1933,0.1036,0.2056


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

Create a final model for prediction.    
Predict labels using test data.

In [ ]:
final_et = finalize_model(tuned_et)
test_pred = predict_model(final_et, data=X_test)

Create a function to evaluate classifier using predicted result and actual test data.


In [14]:
def evaluate_accuracy(y_test, test_pred):
  y_pred = test_pred['Label'].values
  y_pred_1 = np.where(y_pred == 0, 1, 0)
  test_score = test_pred['Score'].values

  prc = classification_report(y_test, y_pred, output_dict=True)
  mc = matthews_corrcoef(y_test, y_pred)
  roc_area = roc_auc_score(y_test, test_score)
  precision, recall, _ = precision_recall_curve(y_test, test_score)
  prc_area = auc(recall, precision)  

  cls = ['0', '1', 'Weighted Avg.']
  tpr = [prc['0']['recall'], prc['1']['recall'], prc['weighted avg']['recall']]
  fpr = [1 - tpr[i] for i in range(len(tpr))]
  pre = [prc['0']['precision'], prc['1']['precision'], prc['weighted avg']['precision']]
  rec = tpr
  fm = [prc['0']['f1-score'], prc['1']['f1-score'], prc['weighted avg']['f1-score']]
  mcc = [mc, mc, mc]
  roc = [roc_area, roc_area, roc_area]
  prc = [prc_area, prc_area, prc_area]
  
  data = {
            'Class': cls,
            'TP Rate': tpr,
            'FP Rate': fpr,
            'Precision': pre,
            'Recall': rec,
            'F-Measure': fm,
            'MCC': mcc,
            'ROC Area': roc,
            'PRC Area': prc
        }
  
  return pd.DataFrame(data)


Evaluate predicted result.

In [ ]:
df_eval = evaluate_accuracy(y_test, test_pred)
df_eval

,Class,TP Rate,FP Rate,Precision,Recall,F-Measure,MCC,ROC Area,PRC Area
0,0,0.586877,0.413123,0.971638,0.586877,0.731763,0.143961,0.418716,0.04484
1,1,0.719298,0.280702,0.096051,0.719298,0.169472,0.143961,0.418716,0.04484
2,Weighted Avg.,0.594493,0.405507,0.921277,0.594493,0.699421,0.143961,0.418716,0.04484


## Attribute selection method

Using the top 5 classifiers obtained from the classifier comparison above, create a function for tuning with training data, making predictions from test data, and evaluating prediction results.

In [23]:
def evaluate_each_clf_accuracy(df_train, X_test, y_test, optimize='Accuracy'):
  clf_model = ['et', 'rf', 'dt', 'lda', 'lightgbm']
  clf_name = ['Extra Trees Classifier', 
              'Random Forest Classifier',
              'Decision Tree Classifier',
              'Linear Discriminant Analysis',
              'Light Gradient Boosting Machine',
              ]
  original_list = ['ballX', 'pitcher_ce', 'batter_ce']
  c_list = [i for i in df_train.columns if i in original_list]
  clf = setup(data=df_train, target='y', session_id=123, 
                categorical_features = c_list, silent = True)  
  df = pd.DataFrame()
  for model, name in zip(clf_model, clf_name):
    m = create_model(model)
    tuned_m = tune_model(m, optimize=optimize)
    final_m = finalize_model(tuned_m)
    test_pred = predict_model(final_m, data=X_test)
    df_new = evaluate_accuracy(y_test, test_pred)
    df_new['Model'] = name
    df = pd.concat([df, df_new])
  
  return df.set_index(["Model", "Class"])

Set the number to select attributes.

In [11]:
k = 5

### Filter method (variance threshold)

In [14]:
threshold = 500000
sel = VarianceThreshold(threshold=threshold)
sel.fit_transform(X_train)
X_new = pd.DataFrame(sel.fit_transform(X_train), columns=X_train.columns.values[sel.get_support()])
print('Before Feature Selection:', X_train.shape)
print('After Feature Selection:', X_new.shape)
new_col = X_train.columns[sel.get_support()]
print('Selected attribute: ', new_col.to_list())

Before Feature Selection: (13463, 42)
After Feature Selection: (13463, 5)
Selected attribute:  ['pitcherHand_ce', 'pitchType_ce', 'ballPositionLabel_ce', 'ballY_ce', 'place_ce']


In [ ]:
df_train_new = X_train[new_col]
df_train_new['y'] = df_train['y']
X_test_new = X_test[new_col]
df_eval = evaluate_each_clf_accuracy(df_train_new, X_test_new, y_test, optimize='MCC')
style = (df_eval.style.highlight_min(color="green").highlight_max(color="red"))  
display(style)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9438,0.6804,0.0,0.0,0.0,0.0,0.0
1,0.9427,0.6368,0.0,0.0,0.0,0.0,0.0
2,0.9427,0.6962,0.0,0.0,0.0,0.0,0.0
3,0.9427,0.7020,0.0,0.0,0.0,0.0,0.0
4,0.9437,0.6500,0.0,0.0,0.0,0.0,0.0
5,0.9437,0.6480,0.0,0.0,0.0,0.0,0.0
6,0.9437,0.7135,0.0,0.0,0.0,0.0,0.0
7,0.9437,0.6520,0.0,0.0,0.0,0.0,0.0
8,0.9437,0.7299,0.0,0.0,0.0,0.0,0.0
9,0.9437,0.6897,0.0,0.0,0.0,0.0,0.0


### Filter method (chi-squared test)

In [16]:
selector = SelectKBest(chi2, k=k)
selector.fit_transform(X_train, y_train)
new_col = X_train.columns[selector.get_support()]
print('Selected attribute: ', new_col.to_list())

Selected attribute:  ['batterHand_ce', 'pitchType_ce', 'ballPositionLabel_ce', 'ballY_ce', 'place_ce']


In [ ]:
df_train_new = X_train[new_col]
df_train_new['y'] = df_train['y']
X_test_new = X_test[new_col]
df_eval = evaluate_each_clf_accuracy(df_train_new, X_test_new, y_test, optimize='MCC')
style = (df_eval.style.highlight_min(color="green").highlight_max(color="red"))  
display(style)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9438,0.6869,0.0,0.0,0.0,0.0,0.0
1,0.9427,0.6413,0.0,0.0,0.0,0.0,0.0
2,0.9427,0.7051,0.0,0.0,0.0,0.0,0.0
3,0.9427,0.7033,0.0,0.0,0.0,0.0,0.0
4,0.9437,0.6387,0.0,0.0,0.0,0.0,0.0
5,0.9437,0.6695,0.0,0.0,0.0,0.0,0.0
6,0.9437,0.7048,0.0,0.0,0.0,0.0,0.0
7,0.9437,0.6502,0.0,0.0,0.0,0.0,0.0
8,0.9437,0.7254,0.0,0.0,0.0,0.0,0.0
9,0.9437,0.6962,0.0,0.0,0.0,0.0,0.0


### Filter method (ANOVA)

In [17]:
selector = SelectKBest(f_classif, k=k)
selector.fit_transform(X_train, y_train)
new_col = X_train.columns[selector.get_support()]
print('Selected attribute: ', new_col.to_list())

Selected attribute:  ['totalPitchingCount', 'B', 'S', 'ballPositionLabel_te', 'ballY_te']


In [ ]:
df_train_new = X_train[new_col]
df_train_new['y'] = df_train['y']
X_test_new = X_test[new_col]
df_eval = evaluate_each_clf_accuracy(df_train_new, X_test_new, y_test, optimize='MCC')
style = (df_eval.style.highlight_min(color="green").highlight_max(color="red"))  
display(style)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9438,0.7518,0.0,0.0,0.0,0.0,0.0
1,0.9427,0.6998,0.0,0.0,0.0,0.0,0.0
2,0.9427,0.7118,0.0,0.0,0.0,0.0,0.0
3,0.9427,0.7313,0.0,0.0,0.0,0.0,0.0
4,0.9437,0.6882,0.0,0.0,0.0,0.0,0.0
5,0.9437,0.6515,0.0,0.0,0.0,0.0,0.0
6,0.9437,0.7014,0.0,0.0,0.0,0.0,0.0
7,0.9437,0.7080,0.0,0.0,0.0,0.0,0.0
8,0.9437,0.7318,0.0,0.0,0.0,0.0,0.0
9,0.9437,0.7032,0.0,0.0,0.0,0.0,0.0


### Filter method (mutual information)

In [18]:
selector = SelectKBest(mutual_info_classif, k=k)
selector.fit_transform(X_train, y_train)
new_col = X_train.columns[selector.get_support()]
print('Selected attribute: ', new_col.to_list())

Selected attribute:  ['ballX', 'ballPositionLabel_ce', 'ballY_ce', 'ballPositionLabel_te', 'ballY_te']


In [ ]:
df_train_new = X_train[new_col]
df_train_new['y'] = df_train['y']
X_test_new = X_test[new_col]
df_eval = evaluate_each_clf_accuracy(df_train_new, X_test_new, y_test, optimize='MCC')
style = (df_eval.style.highlight_min(color="green").highlight_max(color="red"))  
display(style)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9438,0.7068,0.0,0.0,0.0,0.0,0.0
1,0.9427,0.6611,0.0,0.0,0.0,0.0,0.0
2,0.9427,0.7110,0.0,0.0,0.0,0.0,0.0
3,0.9427,0.7225,0.0,0.0,0.0,0.0,0.0
4,0.9437,0.6658,0.0,0.0,0.0,0.0,0.0
5,0.9437,0.6412,0.0,0.0,0.0,0.0,0.0
6,0.9437,0.7225,0.0,0.0,0.0,0.0,0.0
7,0.9437,0.6826,0.0,0.0,0.0,0.0,0.0
8,0.9437,0.7406,0.0,0.0,0.0,0.0,0.0
9,0.9437,0.6784,0.0,0.0,0.0,0.0,0.0


### Wrapper method

Remove attributes with variance of 0

In [9]:
sel = VarianceThreshold()
X = pd.DataFrame(sel.fit_transform(X_train), columns=X_train.columns.values[sel.get_support()])

Use Random Forest and recursive feature elimination

In [12]:
rf = RandomForestClassifier()
selector = RFE(rf, n_features_to_select=k)
selector.fit_transform(X, y_train)
new_col = X.columns[selector.get_support()]
print('Selected attribute: ', new_col.to_list())

Selected attribute:  ['speed', 'batter_ce', 'pitcher_te', 'batter_te', 'ballY_te']


In [16]:
df_train_new = X_train[new_col]
df_train_new['y'] = df_train['y']
X_test_new = X_test[new_col]
df_eval = evaluate_each_clf_accuracy(df_train_new, X_test_new, y_test, optimize='MCC')
style = (df_eval.style.highlight_min(color="green").highlight_max(color="red"))  
display(style)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9311,0.6511,0.0943,0.2273,0.1333,0.1038,0.1148
1,0.9258,0.6338,0.0556,0.1364,0.0789,0.0474,0.0526
2,0.9311,0.7095,0.0926,0.2381,0.1333,0.1046,0.1175
3,0.9215,0.6544,0.0370,0.0833,0.0513,0.0166,0.0181
4,0.9352,0.6308,0.0566,0.2143,0.0896,0.0676,0.0842
5,0.9246,0.6092,0.0189,0.0500,0.0274,-0.0035,-0.0040
6,0.9310,0.6219,0.0377,0.1250,0.0580,0.0327,0.0392
7,0.9416,0.6291,0.0755,0.4000,0.1270,0.1111,0.1545
8,0.9310,0.6073,0.0566,0.1667,0.0845,0.0576,0.0669
9,0.9427,0.7340,0.0943,0.4545,0.1562,0.1396,0.1879


Save train and test data that were used for the best model.

In [27]:
selector = SelectKBest(f_classif, k=k)
selector.fit_transform(X_train, y_train)
new_col_best = X_train.columns[selector.get_support()]
df_train_new = X_train[new_col_best]
X_test_new = X_test[new_col_best]
df_train_new.to_csv('/content/drive/MyDrive/CS699_TermProject/train_data_best.csv', index=False)
X_test_new.to_csv('/content/drive/MyDrive/CS699_TermProject/test_data_best.csv', index=False)